In [1]:
from IPython.display import Markdown, display

display(Markdown("solar PTS.md"))

## Massachusetts RPS / PTS

The Commonwealth of Massachusetts provides data on all of the renewable energy generation facilites, including both commercial and residential, through its website [Lists of Qualified Generation Units](https://www.mass.gov/service-details/lists-of-qualified-generation-units) and directs the user to 7 different programs, see list below.

* [RPS 1](https://www.mass.gov/doc/eligible-class-i-renewable-generation-units-0)
* [SREC I](https://www.mass.gov/files/documents/2017/10/27/Solar%20Carve-Out%20Qualified%20Units%20080717_0.xlsx)
* [SREC II](https://www.mass.gov/doc/solar-carve-out-ii-qualified-renewable-generation-units)
* [SMART](https://www.mass.gov/doc/smart-qualified-units-list)
* [CLASS 2](https://www.mass.gov/doc/rps-class-ii-qualified-units-list-2/download)
* [WASTE](https://www.mass.gov/files/documents/2017/10/27/eligible-class2-waste-units.xls)
* [APS](https://www.mass.gov/doc/aps-qualified-units-list-6)

Verification of monthly production data is centralized through the Massachusetts Clean Energy Center - [Production Tracking System](https://www.masscec.com/about-pts).  PTS data is lagged about 18 months in 2023.  The overlap with the individual *Lists of Qualified Generation Units* is high but with differing data items; see detail and a column naming normalization below.  The [monthly production data](http://files.masscec.com/innovate-clean-energy/prod-track-system/PTSSRECCapacityReport.xlsx) for a large fraction of all systems is available for the years 2010 - 2016 and shows an average 13.71% "capacity"; the proportion of stated kW capacity actually produced.

### Issues

* no unique key
* no (lat,lon) coordinates / location indeterminate



## Set-Up

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv (
        find_dotenv (
            usecwd=True
        ),
    override=True
) # read local .env file and override any existing

from sqlalchemy import create_engine
from os import environ

username     =  environ.get("POSTGRES_USERNAME", "postgres")
password     =  environ.get("POSTGRES_PASSWORD", "postgres")
ipaddress    =  environ.get("POSTGRES_IPADDRESS", "localhost")
port         =  environ.get("POSTGRES_PORT", "5432")
dbname       =  environ.get("POSTGRES_DBNAME", "ArlingtonMA")

#establish database connection for Transform queries and Loads
cnx= create_engine(f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')


In [ ]:
import pandas as pd
import numpy  as np

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick  ##format charts for dollars ($) 

from datetime import datetime, timedelta
import locale  ##format currency for display

locale . setlocale ( locale.LC_ALL , 'en_US.UTF-8' )

##Change this
data_dir = './data/solar'

%matplotlib widget

## Extract

In [ ]:
## Monthly production data from 2010 - 2016 by installation; no key to PTS data
def get_monthly_production_data_2010_2016 ( ) :
    file  =  'http://files.masscec.com/innovate-clean-energy/prod-track-system/PTSSRECCapacityReport.xlsx'

    raw   =  pd . read_excel ( file , sheet_name = 2 )
    raw   =  raw . iloc [ 5 : , : ] #[ spec [ 'skip_rows' ] : , spec [ 'skip_columns' ] : ]
    raw . columns = [ 'year' , 'kW' , 'Jan' , 'Feb' , 'Mar' , 'Apr' , 'May' , 'Jun' , 'Jul' , 'Aug' , 'Sep' , 'Oct' , 'Nov' , 'Dec' , 'total'  , 'cf' ]
    return raw

production_data = get_monthly_production_data_2010_2016 ( )

production_data . groupby ( 'year' ) . agg ( { "cf" : [ np . mean , len ] } )

In [ ]:

norms = {
    'http://files.masscec.com/uploads/attachments/PVinPTSwebsite.xlsx' : {
        'short_name' :  'pts' ,
        'skip_rows'  :  9 ,
        'skip_columns' :   0 ,
        'columns'    :  ['kW','effective_date','cost','grant','city','zip','county','program','type','installer',
                         'module_mfgr','inverter_mfgr','meter_mfgr','utility','owner','srec','est_annual_kWh']
    },
    data_dir + 'Solar Carve-Out Qualified Units 080717_0.xlsx' : {
        'short_name' : 'srec1' ,
        'skip_rows'  :  6 ,
        'skip_columns' : 1 ,
        'columns'    : ['rps_id','nepool_id','aggregation','applicant','name','type','city','zip','kW',
                        'effective_date','operation_date','sq_date','utility','installer','cost','perWatt']
    },
    data_dir + 'RPS Solar Carve-out II Renewable Generation Units_020123.xlsx' : {
        'short_name' : 'srec2' ,
        'skip_rows'  : 12 ,
        'skip_columns' :   0 ,
        'columns'    : ['rps_id','nepool_id','applicant','name','type','city','zip','kW','sector','subsector',
                        'srec_factor','effective_date','operation_date','qualification_date','distributer','installer',
                        'cost','perWatt']
    },
    #data_dir + 'SMART_Solar_Tariff_Generation_Units.xlsx' : {
    data_dir + 'SMART QUL 06-05-2023.xlsx' : {
        'short_name' : 'smart' ,
        'skip_rows'  :  18 ,
        'skip_columns' : 0 ,
        'columns'    : ['project','status','capacity_block','expiration_date','operation_date','effective_date',
                        'distributor','applicant','installer','owner','ownership_type',
                        'type','city','zip','size','kW_ac','kW',
                        'location','location_tranche','off_taker','off_taker_tranche','tracking','tracking_tranche',
                        'pollinator','pollinator_tranche','storage','storage_tranche','storage_kVa','storage_duration',
                        'low_income','land_use','interconnection','meter_type','standalone','cost','perWatt']
    },    
    data_dir + 'RPS Class I Renewable Generation Units.xlsx' : {
        'short_name' : 'rps1' ,
        'skip_rows'  :  16 ,
        'skip_columns' :   0 ,
        'columns'    : ['type','rps_id','nepool_id','name','city','state','fuel','MW','output_percent','aggregator',
                        'verifier','effective_date','qualification_date']
    },
    data_dir + 'Eligible-Class2-Units_20210921.xlsx' : {
        'short_name'   : 'class2' ,
        'skip_rows'    :  17 ,
        'skip_columns' :   1 ,
        'columns'      : ['rps_id','nepool_id','name','city','state','fuel','MW','output_percent',
                        'verifier','effective_date','sq_date']
    },
    data_dir + 'eligible-class2-waste-units.xls' : {
        'short_name'   : 'waste' ,
        'skip_rows'    :  2 ,
        'skip_columns' :   1 ,
        'columns'      : ['rps_id','nepool_id','name','city','state','fuel','MW','sq_date','effective_date','output_percent']
    },
    data_dir + 'APS QUL 1-13-21.xlsx' : {
        'short_name'   : 'aps' ,
        'skip_rows'    :  2 ,
        'skip_columns' :  1 ,
        'columns'    : ['rps_id','nepool_id','name','city','state','fuel','MW','effective_date','operation_date',
                        'rep','verifier']
    },
    
}

In [ ]:
def extract_massgov_rps ( file , spec ):
    
#     raw  =  pd . read_excel ( file )
#     rps  =  raw .iloc [ spec [ 'skip_rows' ] : , spec [ 'skip_columns' ] : ]

    rps  =  pd . read_excel ( file , dtype = { 'Zip' : str , 'zip' : str } , skiprows = spec [ 'skip_rows' ] ) \
               . iloc [ : , spec [ 'skip_columns' ] : ]

    ## assign column names, change the defaults to normalize across datasets
    ### cols =  raw . iloc [ skip_rows - 1 , : ] . tolist ( )
    rps . columns  =  spec [ 'columns' ]

    ## reset index after dropped rows
    rps  =  rps . reset_index ( drop = True )
    
    ## make effective date a pandas datetime index
    mask = rps [ 'effective_date' ] . astype ( str ) . str . contains ( 'Pending|Not Operational' )
    rps . loc [ mask , 'effective_date' ]  =  np.nan
    rps [ 'effective_date' ]  =  pd . to_datetime ( rps [ 'effective_date' ] )

    for col in [ 'kW' , 'cost' , 'est_annual_kWh' ] :
        if col in spec [ 'columns' ] :
            ## 20220615 SMART file exception
            mask = rps [ col ] . isin ( [ '53712.oo' , '37108.5037108.50' ] )
            rps . loc [ mask , col ] = np . nan
            
            rps [ col ] =  rps [ col ] . astype ( np . float64 )
            
    ## MW to floats
    if 'MW' in spec [ 'columns' ] :
        
        if spec [ 'short_name' ] == 'rps1' :
            mask = rps . MW . str . contains ( 'capacity' ) == True
            rps .loc  [ mask, 'MW' ] = '0'
        elif spec [ 'short_name' ] == 'aps' :
            mask = rps . MW . str . contains ( ' thermal ') == True
            rps . loc [ mask , 'MW' ] = 0

        rps . MW = rps . MW . astype ( np . float64 ) 

    if 'state' in spec [ 'columns' ] :
        rps. loc [ : , 'state'] = rps . state . str . upper ( ) . str . replace ( ' ' , '' )        

    return rps 

def exceptions(pts):

    total_installations   = len ( pts . kW )
    total_kW_installed    = pts . kW . sum ( )

    ##150 zero cost thru 4/2021 about 2.2% of total kW installed
    zero_cost = pts [ pts.cost == 0 ] . kW . sum ( )

    ##100 have no install dates, less than .03% of kW installed
    no_install_date = pts [ pd.isnull(pts.effective_date) ] . kW . sum ( )

    ##432 have costs per Watt much too high, 0.15% of kW installed
    high_Per_Watt = pts [ pts.perWatt > 10 ] . kW . sum ( )

    print (
        ( "Exceptions in PTS dataset:\n\n" +\
          "{zc_num} show no cost         totaling {zero_cost}MW comprising {zc_percent}% of total Watt capacity\n"
          "{hi_num} show Per Watt > $10  totaling {high_Per_Watt}MW comprising {hi_percent}% of total Watt capacity\n"
        ) \
        . format (
            total_installations  =  total_installations,
            zc_num               =  len ( pts [ pts.cost == 0 ] ) ,
            zero_cost            =  round ( zero_cost / 1000 , 1 ) ,
            zc_percent           =  round ( 100 * zero_cost / total_kW_installed , 1 ) ,
            hi_num               =  len ( pts [ pts.perWatt > 10 ] ) ,
            high_Per_Watt        =  round ( high_Per_Watt / 1000 , 1 ) ,
            hi_percent           =  round ( 100 * high_Per_Watt / total_kW_installed , 1 ) ,
        )
    )

def PTS_summary ( pts ):
    pts [ 'perWatt' ] = pts . cost . div ( pts . kW ) / 1000

    NM_RATE  = 0.215
    RATE     = 0.11
    url = 'http://files.masscec.com/uploads/attachments/PVinPTSwebsite.xlsx'

    mask = pts [ 'type' ] . str . contains ( 'Residential' )

    #     "and net metered rebates of {nmreturning}M (assuming a constant {nm_rate} rate)**.\n\n" +\
    #     "with an estimated annual retail (net metered) value of {est_annual_value}M ({est_annual_nmval}M)."
    print (
        ("PTS dataset: {url}\n\n" +\
         "There are {count} PV systems ({Rcount} residential) in Massachusetts\n" +\
         "installed between {min_date} and {max_date}\n"  +\
         "by {vendors} unique vendors (22 vendors account for 85% of all installs).\n\n" +\
         "A total of {kW}MW capacity has been installed of which {kW_res}MW is residential\n"    +\
         "at a cost of {invested}M\n" +\
         "generating an estimated {generated}TWh of electricity over the past {years} years\n" +\
         "with  a retail  value of {returning}M (assuming a constant {rate} rate)\n\n" +\
         "Expected energy output from all PV installations is {est_annual}TWh per year\n" +\
         "with an estimated annual retail value of {est_annual_value}M.\n\n"
        ) \
           . format (
        url      =  url ,
        count    =  len ( pts ) ,
        Rcount   =  len ( pts [ mask ] ) ,
        vendors  =  len ( pts . installer . unique ( ) ),
        kW       =  int ( round ( pts . kW . sum ( ) / 1000 , 0 ) ) ,   ## convert kW to MW
        kW_res   =  int ( round ( pts [ mask ] . kW . sum ( ) / 1000 , 0 ) ) ,   ## convert kW to MW
        invested =  locale . currency ( pts . cost . sum ( ) / 1e6 , grouping = True ) , ##convert to millions of $s
        generated = round ( ( ( ( datetime . today ( ) - pts . effective_date ) / timedelta ( days = 365 ) ) * pts.est_annual_kWh ) . sum ( ) / 1e9 , 2 ) , 
        returning   = locale . currency ( RATE * ( ( ( datetime . today ( ) - pts . effective_date ) / timedelta ( days = 365 ) ) * pts . est_annual_kWh ) . sum ( ) / 1e6 , grouping = True ), 
        nmreturning = locale . currency ( NM_RATE * ( ( ( datetime . today ( ) - pts [ mask ] . effective_date ) / timedelta ( days = 365 ) ) * pts [ mask ] . est_annual_kWh ) . sum ( ) / 1e6 , grouping = True ), 
        years = round ( ( pts . effective_date . max ( ) - pts . effective_date . min ( ) ) / timedelta ( days = 365 ) , 1 ) ,
        min_date =  pts . effective_date . min ( ) . strftime( "%Y-%m-%d" ) ,
        max_date =  pts . effective_date . max ( ) . strftime( "%Y-%m-%d" ) ,
        est_annual = round ( ( pts . est_annual_kWh ) . sum ( ) / 1e9 , 2 ) ,
        est_annual_value = locale . currency ( RATE * ( ( pts . est_annual_kWh ) . sum ( ) ) / 1e6 ) ,  ##in millions of $s
        est_annual_nmval = locale . currency ( NM_RATE * ( ( pts [ mask ] . est_annual_kWh ) . sum ( ) ) / 1e6 ) ,  ##in millions of $s
        rate = locale . currency ( RATE ),
        nm_rate = locale . currency ( NM_RATE )
    ) )

    exceptions(pts)
    
    return   


def transform_pts(pts):

    pts_2_db = pts.drop(['zip','county','perWatt'],axis=1).copy()


    pts_2_db=pts_2_db.replace({np.nan:''})
    pts_2_db['W']=(1000*pts_2_db.kW)
    pts_2_db['city']=pts_2_db['city'].replace({'Manchester':'Manchester By The Sea'})

    int_value_pairs = pd.DataFrame()
    for col in ['program', 'type','installer', 'module_mfgr', 'inverter_mfgr', 'meter_mfgr', 'utility','owner','srec']:
        a = list(pts_2_db[col].sort_values().unique())
        pts_2_db[col]=pts_2_db[col].replace(dict(zip(a,range(len(a)))))

        ivp = pd.DataFrame.from_dict(
            dict(zip(range(len(a)),a)),
            orient='index').reset_index().rename(columns={'index':'key',0:'value'})
        ivp['item']='energy_solar_pts_'+col

        int_value_pairs = pd.concat([int_value_pairs,ivp])

    for col in ['cost','grant','est_annual_kWh','W']:
          pts_2_db[col]=pts_2_db[col].astype(int)    

    pts_2_db = pts_2_db.drop(['kW'],axis=1)

    pts_2_db = pts_2_db\
        .merge(dor,how='left',right_on='value',left_on='city')\
        .drop(['city','value'],axis=1)\
        .rename(columns={'effective_date':'date','install_yr':'year','key':'dor'})\
        .sort_values(['dor','date','W','cost'])\
        .reset_index(drop=True)
    
    cols = ['program','type','installer','module_mfgr','inverter_mfgr','meter_mfgr','utility','owner','srec']
    pts_2_db=pts_2_db[['dor','year','date','W','cost','grant','est_annual_kWh']+cols]
    pts_2_db.columns=['dor','year','date','W','cost','grant','est_annual_kWh']+['energy_solar_pts_' + x for x in cols]
    
    return pts_2_db, int_value_pairs


In [ ]:
docs = {}
for file in norms . keys ( ) :   
    docs [ norms [ file ] [ 'short_name' ] ]  = extract_massgov_rps ( file , norms [ file ] )

## 2 installs before 2003, outliers
docs [ 'pts' ] = docs [ 'pts' ] [ docs [ 'pts' ] . effective_date >= '2002-12-17' ]

## remove footers
docs [ 'class2' ]  =  docs [ 'class2' ] . iloc [ : -7 , : ]
docs [ 'rps1'   ]  =  docs [ 'rps1'   ] . iloc [ : -5 , : ]
docs [ 'aps'    ]  =  docs [ 'aps'    ] . iloc [ : -6 , : ]
docs [ 'waste'  ]  =  docs [ 'waste'  ] . iloc [ : -2 , : ]



In [ ]:
pts = docs [ 'pts' ] . copy ( )
PTS_summary ( pts )

## TRANSFORM

In [ ]:
pts = transform_pts(pts)

## LOAD

In [ ]:
table_create_solar_pts = \
    """
        DROP TABLE IF EXISTS energy.solar_pts;
        CREATE TABLE energy.solar_pts (
            "dor" SMALLINT,
            "year" SMALLINT,
            "date" DATE,
            "W" INTEGER,
            "cost" INTEGER,
            "grant" INTEGER,
            "est_annual_kWh" INTEGER,
            "energy_solar_pts_program" SMALLINT,
            "energy_solar_pts_type" SMALLINT,
            "energy_solar_pts_installer" SMALLINT,
            "energy_solar_pts_module_mfgr" SMALLINT,
            "energy_solar_pts_inverter_mfgr" SMALLINT,
            "energy_solar_pts_meter_mfgr" SMALLINT,
            "energy_solar_pts_utility" SMALLINT,
            "energy_solar_pts_owner" SMALLINT,
            "energy_solar_pts_srec" SMALLINT
        );
        
        CREATE INDEX solar_pts_idx 
            ON energy.solar_pts("dor");
        CREATE INDEX solar_pts_idx_year 
            ON energy.solar_pts("year");
        
    """

cnx.execute(table_create_solar_pts)

pts_2_db.to_sql(
    'solar_pts',
    schema='energy',
    con=cnx,
    if_exists='append',
    index=False
) 

int_value_pairs.to_sql(
    'int_value_pairs',
    schema='common',
    con=cnx,
    if_exists='append',
    index=False
) 

## Analysis

In [ ]:
PERCENT_OF_ALL_INSTALLS = 0.85
pts['perWatt'] = pts . cost . div ( pts . kW ) / 1000

mask = (pts['type'].str.contains('Residential')) &\
                                (pts['effective_date']>='2020-01-01') &\
                                (pts['effective_date']<='2021-12-31') &\
                                (pts['cost']>0)   &\
                                (pts['perWatt']<10)  ##note 150 installs have no cost## 432 installs with outlier costs



installers = pts[mask].groupby('installer').agg({
    "kW":[sum,np.median],
    "effective_date":len,
    "perWatt":[np.median,min,max]
})#.sort_values('date',ascending=False)
installers.columns = ['kW_total','kW_median','systems','costPerWatt_median','costPerWatt_min','costPerWatt_max']

for col in ['costPerWatt_median','costPerWatt_min','costPerWatt_max'] :
    installers[col] = installers[col].apply(locale.currency)
    
installers=installers.sort_values('systems',ascending=False)

mask = installers['systems'].cumsum() <= PERCENT_OF_ALL_INSTALLS*len(pts[mask])
print(installers[mask].to_markdown())

In [ ]:
mask = (pts['type'].str.contains('Residential')) &\
                                (pts['effective_date']>='2020-01-01') &\
                                (pts['effective_date']<='2021-12-31')&\
                                (pts['county']=='Middlesex')&\
                                (pts['city']=='Arlington')
installers = pts[mask].groupby('installer').agg({
    "kW":[sum,np.median],
    "effective_date":len,
    "perWatt" : np.median
}).sort_values(("effective_date","len"),ascending=False)
installers.columns = ['total kW installed','median size (kW)','installs','cost_per_Watt']
installers['cost_per_Watt'] = installers['cost_per_Watt'].apply(locale.currency)
installers

In [ ]:
pts [ 'install_yr' ] = pts [ 'effective_date' ] . dt . strftime( '%Y' )

mask = pts [ 'type' ] . str . contains ( 'Residential|residential' )

evolution = pts [ mask ] . groupby ( 'install_yr' ) . agg (
    {
        "kW" : [ sum , np . median , min , max ] ,
        "install_yr" : len ,
        "perWatt" : [ np . median , min , max ] ,
        "est_annual_kWh" : sum
    }
)

evolution . columns = ['kW_sum'         ,
                       'kW_median'      ,
                       'kW_min'         ,
                       'kW_max'         ,
                       'installs'       ,
                       'perWatt_median' ,
                       'perWatt_min'    ,
                       'perWatt_max'    ,
                       'est_annual_kWh'
                      ]


fig, ( ax1 ) = plt . subplots ( ncols = 1 )


x = np . array ( evolution [ 'est_annual_kWh' ] )

ax1 . plot ( 
    evolution [ 'est_annual_kWh' ] . cumsum ( ) / 1e6 , 
    label = 'Cumulative Energy Generated' , 
    lw = 3
)
ax1 . set_title  ( 'MA Residential Solar PV Systems\nAdded Capacity' )
ax1 . set_xlabel ( 'Year Installed' , fontsize = 12 )
ax1 . set_ylabel ( 'Annual GWh Generated (est.)' , fontsize = 12 )

ax = ( evolution [ 'kW_sum' ] / 1000 ) . plot . bar ( 
    secondary_y = True , 
    color = 'g' , 
    label = 'New Power Capacity' ,
    rot = 45
)

ax . set_ylabel ( 'MW', fontsize = 12 )
ax . set_xlabel ( 'Year Installed', fontsize = 12 )

h1, l1 = ax1 . get_legend_handles_labels ( )
h2, l2 = ax  . get_legend_handles_labels ( )


plt . legend ( h1 + h2 , l1 + l2, loc = 'center left' )
plt . show ( )

In [ ]:
%matplotlib widget

fig, ( ax1 ) = plt . subplots ( ncols = 1 )

##note 150 installs have no cost## 432 installs with outlier costs

mask = ( pts [ 'type' ] . str . contains ( 'Residential' ) ) &\
       ( pts [ 'effective_date' ] >= '2018-01-01' )           &\
       ( pts [ 'effective_date' ] <= '2021-12-31' )            &\
       ( pts [ 'cost' ] > 0 )                                   &\
       ( pts [ 'perWatt' ] < 10 ) 


x = np . array ( pts [ mask ] [ 'perWatt' ] )
# filtered = x [ ~is_outlier ( x , 3 ) ]

ax1 . hist ( x , bins = 30 )
ax1 . set_title  ( 'Residential Solar PV Systems\nCost Per Watt Installed\nSince 2018' )
ax1 . set_xlabel ( '' )
ax1 . set_ylabel ( 'Systems' )

fmt   =  '${x:,.2f}'
tick  =  mtick . StrMethodFormatter ( fmt )
ax1 . xaxis . set_major_formatter ( tick )

textstr = '\n'.join((
    r'$\mathrm{average}=\$%.2f$' %(x.mean(),),
    r'$\mathrm{median}=\$%.2f$' %(np.median(x),),
    r'$\sigma=\$%.2f$' %(x.std(),)
))

props = dict(boxstyle='round', facecolor='wheat', alpha = 0.5)
ax1.text(.485, .85, textstr, transform=ax1.transAxes, fontsize=14, verticalalignment='top', bbox = props)

plt.show()